In [1]:
import cv2
import numpy as np

In [2]:
# Inicializar la captura de video
video_capture = cv2.VideoCapture(0)  # 0 para webcam

# Definir rangos de color en formato HSV
COLOR_RANGES = {
    'Rojo': ([0, 100, 100], [10, 255, 255]),
    'Azul': ([100, 100, 100], [130, 255, 255]),
    'Verde': ([40, 100, 100], [80, 255, 255])
}

def get_color(hsv, x, y):
    """Determina el color en la posición (x, y) de la imagen HSV."""
    pixel_color = hsv[y, x]
    for color, (lower, upper) in COLOR_RANGES.items():
        if np.all(np.array(lower) <= pixel_color) and np.all(pixel_color <= np.array(upper)):
            return color
    return "Desconocido"

def get_shape(approx):
    """Determina la forma geométrica según el número de vértices."""
    vertices = len(approx)
    if vertices == 3:
        return "Triángulo"
    elif vertices == 4:
        return "Cuadrado"
    elif vertices >= 8:
        return "Círculo"
    return "Desconocido"

def detect_shapes(frame):
    """Detecta formas geométricas en la imagen y las etiqueta con su color."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    edges = cv2.Canny(blurred, 50, 150)
    
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) < 1000:
            continue
        
        epsilon = 0.03 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)
        
        M = cv2.moments(contour)
        if M["m00"] != 0:
            cX = int(M["m10"] / M["m00"])
            cY = int(M["m01"] / M["m00"])
            color_name = get_color(hsv, cX, cY)
            shape_name = get_shape(approx)
            
            if shape_name != "Desconocido":
                cv2.drawContours(frame, [approx], -1, (0, 255, 0), 3)
                label = f"{shape_name} {color_name}"
                cv2.putText(frame, label, (cX - 20, cY), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
    
    return frame

# Bucle principal
while True:
    ret, frame = video_capture.read()
    if not ret:
        print('Error al leer el fotograma del video')
        break
    
    frame = detect_shapes(frame)
    cv2.imshow('Detección de Formas', frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()